
# Data Engineering Capstone Project

#### Project Summary
The goal of this project is to create a single-source-of-truth date warehouse that can serve as the foundation for creating logical data marts for analytics purposes on I94 immigration data, city temperatures data and U.S. demographics data.    

The project contains the following steps:
1. Scope the Project and Data
2. Explore and Assess the Data
3. Define the Data Model
4. Run ETL to Model the Data
5. Complete Project Write Up

### Import libraries

In [60]:
import pandas as pd
import pyspark
import os
import logging
import datetime as dt

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, monotonically_increasing_id, year, month, to_date, col

### Create Spark Session

In [4]:
spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()

## 1. Project Scope and Data
---

#### Scope 
In order to create a single-source-of-truth data warehouse, the following steps are executed:

* Load dataset into Spark dataframes
* Exploratory data analysis of I94 immigration dataset to identify missing values, empty records, etc, informing the data preprocessing step downstream
* Exploratory data analysis of U.S. city demographics dataset to identify missing values, empty records, etc, informing the data preprocessing step downstream 
* Exploratory data analysis of world temperatures dataset to identify missing values, empty records, etc, informing the data preprocessing step downstream 
* Execute data proprocessing tasks for all datasets
* Create immigration fact table from preprocessed I94 immigration dataset 
* Create dimension tables:
    * Create immigrant demographics dimension table from preprocessed I94 immigration dataset. Relates to immigration fact table by `cic_id` (unique record id) 
    * Create us city demographics dimension table from U.S. city demographics dataset. Relates to immigration fact table by `state_code`
    * Create world temperature dimension table from preprocessed world temperature dataset. Relates to immigration fact table by composite key `city_name`
    * Create country dimension table from `i94cit_i94res` data in the I94_SAS_Labels_Descriptions.SAS file
    * Create city dimension table from `dim_i94port` data in the I94_SAS_Labels_Descriptions.SAS file
    * Create state dimension table from `dim_i94addr` data in I94_SAS_Labels_Descriptions.SAS file
    
##### Datasets:

| Data Set | Format  | Description |
|  :-     |  :-    |  :-        |
|[I94 Immigration Data](https://www.trade.gov/national-travel-and-tourism-office)| SAS | Dataset contains international visitor arrival statistics by world regions, mode of transportation, port of entry, demographics, visa type, etc.|
|[World Temperature Data](https://www.kaggle.com/datasets/berkeleyearth/climate-change-earth-surface-temperature-data)| CSV | Dataset contains monthly average temperatures by city.|
|[U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/)| CSV | Dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.|

##### Tech Stack:
We've made use of the followng technologies in this project: 
- [AWS S3](https://aws.amazon.com/s3/): data storage
- Apache Spark ([PySpark](https://spark.apache.org/docs/latest/api/python/#:~:text=PySpark%20is%20an%20interface%20for,data%20in%20a%20distributed%20environment.)): for reading data from the source (e.g. customer systems / internal systems etc), preprocessing the data and creates fact and dimension tables, and writing the data into fact and dimension tabls on S3.

## 2. Load and Preprocess Data

* Load datasets into Spark dataframes
* Remove duplicates, empty rows and columns with significant amount of missing values (>85%)

### I94 Immigration Data
---

#### Load I94 Immigration Data

In [14]:
# Load data for April 2016
file_name = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immi_raw = spark.read.format('com.github.saurfang.sas.spark').load(file_name)

In [15]:
# Number of records
df_immi_raw.count()

3096313

In [16]:
# Schema
df_immi_raw.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [17]:
# Display top five records
df_immi_raw.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


##### Data dictionary

| Field Name | Description |
|  :-      |  :-        |
| cicid    | Unique record ID |
|i94yr     | 4  digit year|
|i94mon| Numeric month |
|i94cit|3 digit code for immigrant country of birth|
|i94res|3 digit code for immigrant country of residence|
|i94port|Port of admission|
|arrdate|Arrival Date in the USA|
|i94mode|Mode of transportation (1 = Air; 2 = Sea; 3 = Land; 9 = Not reported)|
|i94addr|USA State of arrival|
|depdate|Departure Date from the USA|
|i94bir|Age of Respondent in Years|
|i94visa|Visa codes collapsed into three categories|
|count|Field used for summary statistics|
|dtadfile|Character Date Field - Date added to I-94 Files|
|visapost|Department of State where where Visa was issued|
|occup|Occupation that will be performed in U.S|
|entdepa|Arrival Flag - admitted or paroled into the U.S.|
|entdepd|Departure Flag - Departed, lost I-94 or is deceased|
|entdepu|Update Flag - Either apprehended, overstayed, adjusted to perm residence|
|matflag|Match flag - Match of arrival and departure records|
|biryear|4 digit year of birth|
|dtaddto|Character Date Field - Date to which admitted to U.S. (allowed to stay until)|
|gender|Non-immigrant sex|
|insnum|INS number|
|airline|Airline used to arrive in U.S.|
|admnum|Admission Number|
|fltno|Flight number of Airline used to arrive in U.S.|
|visatype|Class of admission legally admitting the non-immigrant to temporarily stay in U.S.|

#### Preprocess I94 Immigration Data

In [18]:
# Columns with +85% missing values (as identified in EDA notebook)
cols = ['entdepu', 'occup', 'insnum']

# Drop columns
df_immi_clean = df_immi_raw.drop(*cols)

In [19]:
# Schema
df_immi_clean.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [22]:
# Drop duplicates on unique identifier 'cicid'
df_immi_clean = df_immi_clean.dropDuplicates(['cicid'])
df_immi_clean.count()

3096313

In [23]:
# Drop records missing unique identifier 'cicid'
df_immi_clean = df_immi_clean.dropna(how='all', subset=['cicid'])
df_immi_clean.count()

3096313

In [39]:
def preprocess_immigration_data(df):
    """Preprocess immigration dataframe
    :param df: spark dataframe with immigration data
    :return: spark dataframe with preprocessed immigration data
    """    
    logging.info(f'Total records in raw dataframe: {df.count()}')
    print(f'Total records in raw dataframe: {df.count()}')
    
    # Remove columns with +85% missing values as identified during EDA
    drop_columns = ['entdepu', 'occup', 'insnum']    
    df = df.drop(*drop_columns)
    
    # Remove duplicate records on 'cicid'
    df = df.dropDuplicates(['cicid'])
    
    # Remove empty records
    df = df.dropna(how='all')
    
    logging.info(f'Total records in preprocessed dataframe: {df.count()}')
    print(f'Total records in preprocessed dataframe: {df.count()}')
    
    return df

In [40]:
df_immi_clean = preprocess_immigration_data(df_immi_raw)

Total records in raw dataframe: 3096313
Total records in preprocessed dataframe: 3096313


### World Temperature data
---

#### Load World Temperature Data

In [52]:
file_name = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temperature_raw = spark.read.csv(file_name, header=True, inferSchema=True)

In [53]:
# Number of records
df_temperature_raw.count()

8599212

In [54]:
# Schema
df_temperature_raw.printSchema()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [55]:
# Display top five records
df_temperature_raw.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


##### Data dictionary
| Field Name | Description |
|  :-      |  :-        |
|dt|Date|
|AverageTemperature|Global average city temperature in celsius|
|AverageTemperatureUncertainty|95% confidence interval around the average|
|City|Name of city|
|Country|Name of country|
|Latitude|City latitude|
|Longitude|City longitude|

#### Preprocess World Temperature Data

In [56]:
def preprocess_temperature_data(df):
    """Preprocess world temperature dataset to remove records with missing values and duplicates
    
    :param df: spark dataframe with world temperature data
    :return: spark dataframe with preprocessed world temperature data
    """
    logging.info(f'Total records in raw dataframe: {df.count()}')
    print(f'Total records in raw dataframe: {df.count()}')
    
    # Remove records with missing average temperature
    df = df.dropna(subset=['AverageTemperature'])
    
    # Remove duplicate records on date, city and country
    df = df.dropDuplicates(subset=['dt', 'City', 'Country'])
    
    # Remove empty rows
    df = df.dropna(how='all')
    
    logging.info(f'Total records in preprocessed dataframe: {df.count()}')
    print(f'Total records in preprocessed dataframe: {df.count()}')
    
    return df

In [57]:
df_temperature_clean = preprocess_temperature_data(df_temperature_raw)

Total records in raw dataframe: 8599212
Total records in preprocessed dataframe: 8190783


### U.S. City Demographic data
---

#### Load U.S. City Demographics Data

In [4]:
file_name = "us-cities-demographics.csv"
df_demo_raw = spark.read.csv(file_name, inferSchema=True, header=True, sep=';')

In [5]:
# Number of records
df_demo_raw.count()

2891

In [46]:
# Schema
df_demo_raw.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [47]:
# Display top five records
df_demo_raw.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


##### Data dictionary
| Field Name | Description |
|  :-      |  :-        |
|City|City Name|
|State|US State where city is located|
|Median Age|Median age of the population|
|Male Population|Count of male population|
|Female Population|Count of female population|
|Total Population|Count of total population|
|Number of Veterans|Count of total Veterans|
|Foreign born|Count of residents of the city that were not born in the city|
|Average Household Size|Average city household size|
|State Code|Code of the US state|
|Race|Respondent race|
|Count|Count of city's individual per race|

#### Preprocess U.S. Demographics Data

In [48]:
def preprocess_demographics_data(df):
    """Preprocess US demographics dataset to remove records with missing values and duplicates
    
    :param df: spark dataframe with us demograpgics data
    :return: spark dataframe with processed us demograpgics data
    """
    logging.info(f'Total records in raw dataframe: {df.count()}')
    print(f'Total records in raw dataframe: {df.count()}')
    
    # Remove duplicate records on city, state and race
    df = df.dropDuplicates(subset=['City', 'State', 'Race'])
    
    # Remove empty rows
    df.dropna(how="all")
    
    logging.info(f'Total records in preprocessed dataframe: {df.count()}')
    print(f'Total records in preprocessed dataframe: {df.count()}')
    
    return df

In [50]:
df_demo_clean = preprocess_demographics_data(df_demo_raw)

Total records in raw dataframe: 2891
Total records in preprocessed dataframe: 2891


### Airport Code data
---

#### Load U.S. City Demographics Data

In [11]:
file_name = "airport-codes_csv.csv"
df_airport_raw = spark.read.csv(file_name, inferSchema=True, header=True)

In [12]:
# Number of records
df_airport_raw.count()

55075

In [13]:
# Schema
df_airport_raw.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [14]:
# Display top five records
df_airport_raw.limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


##### Data dictionary
| Field Name | Description |
|  :-      |  :-        |
|ident| Unique ID |
|type |Type of airport|
|name |Airport name|
|elevation_ft |Airport elevation in feet|
|Continent | Continent|
|iso_country| ISO country code|
|iso_region|ISO region code|
|municipality|Municipality name|
|gps_code | GPS code|
|iata_code|Three-character alphanumeric geocode designating airport |
|local_code| Local code|
|coordinates| Airport Longitude and Latitude|

## 3. Define the Data Model
#### 3.1 Conceptual Data Model
The conceptual data model for our single-source-of-truth datawarehouse looks as follows:

<img src="erd_data_warehouse.png" alt="Conceptual model" width="1500" height="1500" />

#### 3.2 Mapping Out Data Pipelines
The data pipeline is as follows:

1. Load datasets stored in S3 buckets into Spark dataframes:
    - [Source_S3_Bucket]/immigration_data/18-83510-I94-Data-2016/*.sas7bdat
    - [Source_S3_Bucket]/I94_SAS_labels_data/I94_SAS_Labels_Descriptions.SAS
    - [Source_S3_Bucket]/temperature_data/GlobalLandTemperaturesByCity.csv
    - [Source_S3_Bucket]/us_demographics_data/us-cities-demographics.csv
    

2. Create helper dimension tables from I94_SAS_Labels_Descriptinons.SAS file
    - Create country dimension table from `i94cit_i94res` data in the I94_SAS_Labels_Descriptions.SAS file
    - Create city dimension table from `dim_i94port` data in the I94_SAS_Labels_Descriptions.SAS file
    - Create state dimension table from `dim_i94addr` data in I94_SAS_Labels_Descriptions.SAS file

3. Preprocess I94 Immigration data
4. Create I94 Immigration fact table - `fact_immigration` - from preprocessed I94 Immigration data  
5. Create I94 Immigration demographics dimension table - `dim_immigrant_demographics` - from preprocessed I94 Immigration data 
6. Create U.S. City Demographic dimension table - `dim_city_demographics` - from U.S. City Demographic data
7. Preprocess World Temperature data
8. Create World Temperature dimension table - `dim_city_temperature` - from preprocessed World Temperature data 

## 4. Run Pipelines to Model the Data 

#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [6]:
# Specify output path (target bucket on S3)
output_data = 'tables/'

##### Create Immigration fact table
---

In [72]:
def create_immigration_fact_table(df, output_data):
    """Creates an immigration fact table from  I94 Immigration data.
    
    :param df: spark dataframe with preprocessed immigration data
    :param output_data: write path
    :return: spark dataframe with immigration fact data
    """    
    # UDF to convert SAS date format to datetime object
    get_datetime = udf(lambda x: (dt.datetime(1960, 1, 1).date() + dt.timedelta(x)).isoformat() if x else None)
    
    df = df.withColumnRenamed('cicid', 'cic_id') \
           .withColumnRenamed('i94yr', 'year') \
           .withColumnRenamed('i94mon', 'month') \
           .withColumnRenamed('i94port', 'city_code') \
           .withColumnRenamed('i94addr', 'state_code') \
           .withColumnRenamed('arrdate', 'arrival_date') \
           .withColumnRenamed('depdate', 'departure_date') \
           .withColumnRenamed('i94mode', 'mode') \
           .withColumnRenamed('i94visa', 'visa') \
           .withColumnRenamed('visatype', 'visa_type')
    
    # convert dates into datetime objects
    df = df.withColumn('arrival_date', get_datetime(df.arrival_date))
    df = df.withColumn('departure_date', get_datetime(df.departure_date))
    
    df = df.withColumn('immigration_id', monotonically_increasing_id())
    
    # write fact table to parquet file partioned by state
    df.write.mode('overwrite').partitionBy('state_code').parquet(path=output_data + "fact_immigration")
    
    return df

In [73]:
immi_fact_df = create_immigration_fact_table(df_immi_clean, output_data)
immi_fact_df.limit(5).toPandas()

,cic_id,year,month,i94cit,i94res,city_code,arrival_date,mode,state_code,departure_date,...,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visa_type,immigration_id
0,299.0,2016.0,4.0,103.0,103.0,NYC,2016-04-01,1.0,NY,2016-04-06,...,O,M,1962.0,06292016,None,OS,5.542587e+10,00087,WT,0
1,305.0,2016.0,4.0,103.0,103.0,NYC,2016-04-01,1.0,NY,2016-04-11,...,O,M,1953.0,06292016,None,OS,5.542582e+10,00087,WT,1
2,496.0,2016.0,4.0,103.0,103.0,CHI,2016-04-01,1.0,IL,2016-04-04,...,O,M,1952.0,06292016,None,OS,5.542862e+10,00065,WB,2
3,558.0,2016.0,4.0,103.0,103.0,SFR,2016-04-01,1.0,CA,2016-04-03,...,O,M,1974.0,06292016,M,LH,5.543331e+10,00454,WB,3
4,596.0,2016.0,4.0,103.0,103.0,NAS,2016-04-01,1.0,FL,2016-04-03,...,N,M,1992.0,06292016,M,UP,5.540611e+10,00221,WT,4


##### Create Immigrant Demographics dimension table
---

In [74]:
def create_immi_demographics_dim_table(df, output_data):
    """Creates an immigrant demographics dim table from  I94 Immigration data.
    
    :param df: spark dataframe of immigration data
    :param output_data: write path
    :return: spark dataframe with immigrant demographics fact data
    """    
    df = df.withColumnRenamed('cicid','cic_id') \
           .withColumnRenamed('i94cit', 'country_of_birth') \
           .withColumnRenamed('i94res', 'country_of_residence') \
           .withColumnRenamed('biryear', 'year_of_birth') \
           .withColumnRenamed('ins_num', 'insnum')
    
    df = df.withColumn('immi_demographics_id', monotonically_increasing_id())
    
    # write dimension table to parquet file
    df.write.mode('overwrite').parquet(path=output_data + "dim_immigrant_demographics")
    
    return df

In [75]:
immi_demo_dim_df = create_immi_demographics_dim_table(df_immi_clean, output_data)
immi_demo_dim_df.limit(5).toPandas()

,cic_id,i94yr,i94mon,country_of_birth,country_of_residence,i94port,arrdate,i94mode,i94addr,depdate,...,entdepd,matflag,year_of_birth,dtaddto,gender,airline,admnum,fltno,visatype,immi_demographics_id
0,299.0,2016.0,4.0,103.0,103.0,NYC,20545.0,1.0,NY,20550.0,...,O,M,1962.0,06292016,None,OS,5.542587e+10,00087,WT,0
1,305.0,2016.0,4.0,103.0,103.0,NYC,20545.0,1.0,NY,20555.0,...,O,M,1953.0,06292016,None,OS,5.542582e+10,00087,WT,1
2,496.0,2016.0,4.0,103.0,103.0,CHI,20545.0,1.0,IL,20548.0,...,O,M,1952.0,06292016,None,OS,5.542862e+10,00065,WB,2
3,558.0,2016.0,4.0,103.0,103.0,SFR,20545.0,1.0,CA,20547.0,...,O,M,1974.0,06292016,M,LH,5.543331e+10,00454,WB,3
4,596.0,2016.0,4.0,103.0,103.0,NAS,20545.0,1.0,FL,20547.0,...,N,M,1992.0,06292016,M,UP,5.540611e+10,00221,WT,4


##### Create City Demographics dimension table
---

In [78]:
def create_city_demographics_dimension_table(df, output_data):
    """Creates a us city demographics dimension table from the U.S. City Demographic dataset.
    
    :param df: spark dataframe of us city demographics data
    :param output_data: write path
    :return: spark dataframe with demographics data
    """
    df = df.withColumnRenamed('City', 'city_code') \
           .withColumnRenamed('State Code', 'state_code') \
           .withColumnRenamed('Median Age','median_age') \
           .withColumnRenamed('Male Population', 'male_population') \
           .withColumnRenamed('Female Population', 'female_population') \
           .withColumnRenamed('Total Population', 'total_population') \
           .withColumnRenamed('Number of Veterans', 'number_of_veterans') \
           .withColumnRenamed('Foreign-born', 'foreign_born_num') \
           .withColumnRenamed('Average Household Size', 'avg_household_size') \
           .withColumnRenamed('Race', 'race') \
           .withColumnRenamed('Count', 'count')

    df = df.withColumn('id', monotonically_increasing_id())
    
    # write dimension table to parquet file
    df.write.mode('overwrite').parquet(path=output_data + "dim_city_demographics")
    
    return df

In [79]:
demo_dim_df = create_city_demographics_dimension_table(df_demo_clean, output_data)
demo_dim_df.limit(5).toPandas()

,city_code,State,median_age,male_population,female_population,total_population,number_of_veterans,foreign_born_num,avg_household_size,state_code,race,count,id
0,Alafaya,Florida,33.5,39504,45760,85264,4176,15842,2.94,FL,White,63666,0
1,Baldwin Park,California,35.8,38747,38309,77056,780,34322,4.13,CA,Black or African-American,1560,1
2,Houston,Texas,32.6,1149686,1148942,2298628,71898,696210,2.66,TX,Asian,173854,2
3,Las Cruces,New Mexico,32.7,47835,53809,101644,9421,11888,2.58,NM,White,91201,3
4,Missouri City,Texas,37.2,34932,36846,71778,4274,18556,3.03,TX,Asian,17854,4


##### Create Global Temperature dimension table
---

In [67]:
def create_temperature_dimension_table(df, output_data):
    """Creates a global temperature dimension table from the Global Average Temperature dataset.
    
    :param df: spark dataframe of global average temperature by city data
    :param output_data: write path
    :return: spark dataframe with average temperature by city data
    """
    df = df.withColumn('dt', to_date(col('dt'))) \
           .withColumnRenamed('City', 'city_name') \
           .withColumnRenamed('Country', 'country_name') \
           .withColumnRenamed('AverageTemperature','avg_temperature') \
           .withColumnRenamed('AverageTemperatureUncertainty', 'avg_temperature_delta')
    
    # Derive month and year from datetime column 
    df = df.withColumn('year', year(df['dt']))
    df = df.withColumn('month', month(df['dt']))

    # write dimension table to parquet file
    df.write.mode('overwrite').parquet(path=output_data + "dim_temperature")
    
    return df

In [68]:
temperature_dim_df = create_temperature_dimension_table(df_temperature_clean, output_data)
temperature_dim_df.limit(5).toPandas()

,dt,avg_temperature,avg_temperature_delta,city_name,country_name,Latitude,Longitude,year,month
0,1743-11-01,5.354,1.636,Frankfurt,Germany,50.63N,8.87E,1743,11
1,1743-11-01,7.030,1.611,Münster,Germany,52.24N,7.88E,1743,11
2,1743-11-01,5.044,2.222,Pitesti,Romania,44.20N,24.60E,1743,11
3,1743-11-01,13.808,1.918,Sabadell,Spain,40.99N,2.13E,1743,11
4,1744-04-01,10.352,2.222,Akron,United States,40.99N,80.95W,1744,4


### I94 SAS Labels Descriptions
---

Create `dim_country` table from `i94cit_i94res` data in I94_SAS_Labels_Descriptinons.SAS file

In [39]:
i94cit_i94res = {}
for countries in contents[9:298]:
    pair = countries.split('=')
    country_code, country_name = pair[0].strip(), pair[1].strip().strip("'")
    i94cit_i94res[country_code] = country_name

In [40]:
df_i94cit_i94res = pd.DataFrame(list(i94cit_i94res.items()), columns=['country_code', 'country_name'])
df_i94cit_i94res.head(5)

,country_code,country_name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [41]:
spark.createDataFrame(i94cit_i94res.items(), ['country_code', 'country_name'])\
    .write.mode("overwrite")\
    .parquet(path=output_data + 'dim_country')

Create `dim_city` table from `dim_i94port` data in I94_SAS_Labels_Descriptinons.SAS file

In [42]:
i94port = {}
for cities in contents[302:962]:
    pair = cities.split('=')
    city_code, city_name = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip().strip("''")
    i94port[city_code] = city_name

In [43]:
df_i94port = pd.DataFrame(list(i94port.items()), columns=['city_code', 'city_name'])

In [44]:
df_i94port[['city_name', 'state_code']] = df_i94port['city_name'].str.split(',', 1, expand=True)
df_i94port['city_name'] = df_i94port['city_name'].str.title()
df_i94port.drop(columns='state_code', inplace=True)

In [45]:
spark.createDataFrame(df_i94port)\
    .write.mode("overwrite")\
    .parquet(path=output_data + 'dim_city')

In [46]:
df_i94port.head()

,city_code,city_name
0,ALC,Alcan
1,ANC,Anchorage
2,BAR,Baker Aaf - Baker Island
3,DAC,Daltons Cache
4,PIZ,Dew Station Pt Lay Dew


Create `dim_state` table from `dim_i94addr` data in I94_SAS_Labels_Descriptinons.SAS file

In [47]:
i94addr = {}
for states in contents[981:1036]:
    pair = states.split('=')
    state_code, state_name = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    i94addr[state_code] = state_name.title()

In [48]:
df_i94addr = pd.DataFrame(list(i94addr.items()), columns=['state_code', 'state_name'])
df_i94addr.head()

,state_code,state_name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [49]:
spark.createDataFrame(df_i94addr)\
    .write.mode("overwrite")\
    .parquet(path=output_data + 'dim_state')

#### 4.2 Data Quality Checks
Run data quality check for data completened: verify ETL process has created and populated fact and dimension tables

In [80]:
def run_quality_check(df, table_name):
    """Check for non-empty fact and dimension tables.
    :param df: spark dataframe
    :param table_name: table name
    """
    total_count = df.count()

    if total_count == 0:
        print(f"Data quality check FAILED for {table_name}: no records found!")
        logging.warning(f"Data quality check FAILED for {table_name}: no records found!")
    else:
        print(f"Data quality check PASSED for {table_name}: {total_count} records found!")
        logging.info(f"Data quality check PASSED for {table_name}: {total_count} records found!")

In [81]:
tables_dict = {
    'immigration_fact': immi_fact_df,
    'immigrant_demographics_dim': immi_demo_dim_df,
    'demographics_dim': demo_dim_df
}

for table_name, df in tables_dict.items():
    run_quality_check(df, table_name)

Data quality check passed for immigration_fact: 3096313 records found!
Data quality check passed for immigrant_demographics_dim: 3096313 records found!
Data quality check passed for demographics_dim: 2891 records found!


#### 4.3 Data Dictionary of the Data Model 


<img src="data_dictionary.png" width="1500" height="1500" />

#### 5. Project Write Up

##### 5.1 The rationale for the chosen tools and technologies
* [AWS S3](https://aws.amazon.com/s3/) for data storage.
* Apache Spark ([PySpark](https://spark.apache.org/docs/latest/api/python/#:~:text=PySpark%20is%20an%20interface%20for,data%20in%20a%20distributed%20environment.)) processing the data and creating fact and dimension tables.

##### 5.2 Data update frequency
* The immigration fact and immigrant demographics dimension table, and temperature table should be updated on a monthly schedule as the raw data is aggregated on a monthly time period.
* The US city demographics table can be updated depending on the refresh time period of the raw data, which, given how involved it is to update census data, probably annually.

##### 5.3 Future work
5.3.1 The data was increased by 100x
* It seems unlikely that a 100x increase in the data size would be efficiently processes by Apache Spark in standalone server mode and a cloud big data plaform for running large-scale distributed processing jobs such as [Amazon EMR](https://aws.amazon.com/emr/) should be considered to scale.

5.3.2 The data populates a dashboard that must be updated on a daily basis by 7am every day.
* [Apache Airflow](https://airflow.apache.org/) can be used for building out an ETL data pipeline that automates the tasks of processing fresh data and updating the dashboard on a daily basis by 7am.   
 
5.3.3 The database needed to be accessed by 100+ people.
* In this scenario we would move our single-source-of-truth database to a cloud dataware house such as [Amazon Redshift](https://aws.amazon.com/redshift/).